In [1]:
instagram_features = """The Instagram in-app camera has a variety of features that allow you to take and share photos and videos in a creative and engaging way. Some of these features include:

Filters: Instagram has a wide variety of filters that you can apply to your photos and videos to change their look and feel. These filters can be used to adjust the brightness, contrast, saturation, and other aspects of your photos, as well as to add special effects like vintage, black and white, or retro.
Stickers: Instagram also offers a variety of stickers that you can add to your photos and videos. These stickers can be used to add text, emojis, location information, and other elements to your content.
Reels: Reels are a type of short-form video that can be created using the Instagram in-app camera. Reels can be up to 60 seconds long and can include music, filters, stickers, and other effects.
Live: Live is a feature that allows you to broadcast live video to your followers. This is a great way to connect with your audience in real time and share your experiences with them.
Dual camera: The dual camera feature allows you to record video with both the front and rear cameras at the same time. This is a great way to create more engaging content, such as vlogs or reaction videos.
These are just a few of the many features that are available in the Instagram in-app camera. With so many options available, you can be sure to find the perfect way to express yourself and share your creativity with the world.

Here are some additional features of the Instagram in-app camera:

Superzoom: This feature allows you to zoom in on your subject while recording a video, creating a dramatic effect.
Boomerang: This feature records a short video that plays forward and then backward, creating a looping effect.
Layout: This feature allows you to combine multiple photos or videos into a single post.
Story stickers: These stickers are specific to Stories and can be used to add interactive elements, such as polls, quizzes, and countdowns.
AR effects: These effects use augmented reality to overlay digital objects onto your photos and videos.
The Instagram in-app camera is constantly being updated with new features, so be sure to check back often to see what's new."""

snapchat_features = """The Snapchat in-app camera has a variety of features that allow you to take and share photos and videos in a creative and engaging way. Some of these features include:

Lenses: Snapchat is known for its Lenses, which are augmented reality filters that can be applied to your photos and videos. Lenses can be used to change your appearance, add funny props, or interact with the world around you.
Filters: Snapchat also has a variety of filters that you can apply to your photos and videos. These filters can be used to adjust the brightness, contrast, saturation, and other aspects of your photos, as well as to add special effects like vintage, black and white, or retro.
Bitmoji: Snapchat users can create their own Bitmoji, which is a personalized cartoon avatar. Bitmoji can be used in a variety of ways, such as as stickers, in Lenses, or as your profile picture.
Stories: Snapchat Stories are a way to share photos and videos that disappear after 24 hours. Stories are a great way to share your day with your friends and followers, and they can also be used to promote your brand or business.
Chat: Snapchat Chat is a way to send one-on-one or group messages. Chat messages can include text, photos, videos, and Lenses.
Discover: Snapchat Discover is a section of the app that features content from publishers, creators, and brands. Discover is a great way to find new content and stay up-to-date on the latest trends.
These are just a few of the many features that are available in the Snapchat in-app camera. With so many options available, you can be sure to find the perfect way to express yourself and share your creativity with the world.

Here are some additional features of the Snapchat in-app camera:

Geofilters: These filters are specific to a location and can be used to add a fun or informative element to your photos and videos.
Quick Chat: This feature allows you to send pre-written messages to your friends.
Snap Map: This feature shows you where your friends are located and lets you share your own location.
Memories: This feature allows you to save your Snaps and Stories so you can access them later.
Snapcodes: These are unique QR codes that can be scanned to add friends or open links.
The Snapchat in-app camera is constantly being updated with new features, so be sure to check back often to see what's new."""

In [2]:
import langchain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator

In [52]:
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 50,
    length_function = len,
)

# token_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=100, chunk_overlap=0
# )

texts = text_splitter.create_documents([instagram_features, snapchat_features], metadatas=[{"source": "instagram"}, {"source": "snapchat"}])

# tokens_instagram = token_splitter.split_text(instagram_features)
# tokens_snapchat = token_splitter.split_text(snapchat_features)

Created a chunk of size 1323, which is longer than the specified 1000
Created a chunk of size 1481, which is longer than the specified 1000


In [53]:
embeddings = OpenAIEmbeddings()

text_db = Chroma.from_documents(texts, embeddings)

# token_db_instagram = Chroma.from_texts(tokens_instagram, embeddings)
# token_db_snapchat = Chroma.from_texts(tokens_snapchat, embeddings)

In [54]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)
from langchain.llms import OpenAI

llm = OpenAI(
    model="text-davinci-003",
    temperature=0,
)

vectorstore_info = VectorStoreInfo(
    name="text_db",
    description="instagram and snapchat features",
    vectorstore=text_db,
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(llm=llm, toolkit=toolkit, verbose=True)

In [50]:
agent_executor.run(
    "list some features of the instagram in-app camera"
)



> Entering new AgentExecutor chain...
 I need to use a tool to answer this
Action: text_db
Action Input: features of the instagram in-app camera
Observation:  Superzoom, Boomerang, Layout, Story stickers, AR effects, Filters, Stickers, Reels, Live, Dual camera.
Thought: I now know the final answer
Final Answer: Superzoom, Boomerang, Layout, Story stickers, AR effects, Filters, Stickers, Reels, Live, Dual camera.

> Finished chain.


'Superzoom, Boomerang, Layout, Story stickers, AR effects, Filters, Stickers, Reels, Live, Dual camera.'

In [55]:
agent_executor.run("Compare the features of the instagram in-app camera and the snapchat in-app camera")



> Entering new AgentExecutor chain...
 I need to use a tool to answer this question
Action: text_db_with_sources
Action Input: Compare the features of the instagram in-app camera and the snapchat in-app camera
Observation: {"answer": " The Snapchat in-app camera features include Geofilters, Quick Chat, Snap Map, Memories, and Snapcodes. The Instagram in-app camera features include Superzoom, Boomerang, Layout, Story stickers, and AR effects.\n", "sources": "snapchat, instagram"}
Thought: I now know the final answer
Final Answer: The Snapchat in-app camera features include Geofilters, Quick Chat, Snap Map, Memories, and Snapcodes. The Instagram in-app camera features include Superzoom, Boomerang, Layout, Story stickers, and AR effects.

> Finished chain.


'The Snapchat in-app camera features include Geofilters, Quick Chat, Snap Map, Memories, and Snapcodes. The Instagram in-app camera features include Superzoom, Boomerang, Layout, Story stickers, and AR effects.'